Imports

In [1]:
import feed_forward
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
import helper
torch.cuda.is_available()

True

CUDA check

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Data Transformation

In [3]:
transform = transforms.Compose([transforms.ToTensor()
                              ])

Training and Test data from MNIST data set

In [4]:
trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
#print(trainset[1])
valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=512, shuffle=True)

C:\Users\julia\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Initializing the model

In [5]:
model = feed_forward.HypFF()
model.to(device)

HypFF(
  (fc1): MobLinear(
    in_features=784, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=512, out_features=256, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=256, out_features=10, bias=True
    (ball): PoincareBall manifold
  )
)

In [6]:
print(model)

HypFF(
  (fc1): MobLinear(
    in_features=784, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=512, out_features=256, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=256, out_features=10, bias=True
    (ball): PoincareBall manifold
  )
)


Hyperparameters

In [7]:
learning_rate = 16e-4
momentum = 0.9

Loss Function

In [ ]:
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(trainloader))
images, labels = images.to(device), labels.to(device)
images = images.view(images.shape[0], -1)

out = model(images) #output
print(out)
loss = criterion(out, labels) #calculate the loss

Optimizer

In [9]:
optimizer = geoopt.optim.RiemannianSGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [10]:

time0 = time()

epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #backpropagation
        loss.backward()
        
        #Weight optimization
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 2.172604267880068
Epoch 1 - Training loss: 1.9417641415434368
Epoch 2 - Training loss: 1.839470708774308
Epoch 3 - Training loss: 1.793097715256578
Epoch 4 - Training loss: 1.7647157786256176
Epoch 5 - Training loss: 1.745619189941277
Epoch 6 - Training loss: 1.731230513524201
Epoch 7 - Training loss: 1.720386853662588
Epoch 8 - Training loss: 1.7118488166291834
Epoch 9 - Training loss: 1.7051353323257576
Epoch 10 - Training loss: 1.6993606828026853
Epoch 11 - Training loss: 1.694609364210549
Epoch 12 - Training loss: 1.690422964298119
Epoch 13 - Training loss: 1.6868758848157979
Epoch 14 - Training loss: 1.684162890507003

Training Time (in minutes) = 3.1934517780939737


In [11]:
images, labels = next(iter(valloader))
#images, labels = images.to(device), labels.to(device)

img = images[0].view(1, 784)
img = img.to(device)
with torch.no_grad():
    out = model(img)

ps = out.cpu()
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))
#helper.view_classify(img.view(1, 28, 28), ps)

Predicted Digit = 0


In [25]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  images, labels = images.to(device), labels.to(device)
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        out = model(img)

    
    ps = out.cpu()
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu().numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))